In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import codecs
from SmilesPE.tokenizer import *
from collections import Counter
from deepchem.feat.smiles_tokenizer import SmilesTokenizer
import os
from sklearn.preprocessing import LabelEncoder

/Users/jayrussell/opt/anaconda3/envs/tflow/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/jayrussell/opt/anaconda3/envs/tflow/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Drug to Drug interactions (DDI)
Curating a data set with confirmed drug to drug interactions + a list of their described chemical structures.

In [3]:
#Interaction involving alimentary tract and metabolism drugs
alimentary = pd.read_csv('../data/drug interactions/ddinter_downloads_code_A.csv')
#Interaction involving blood and blood forming organs drugs
blood = pd.read_csv('../data/drug interactions/ddinter_downloads_code_B.csv')
#Interaction involving dermatological drugs
derma = pd.read_csv('../data/drug interactions/ddinter_downloads_code_D.csv')
#Interaction involving systemic hormonal preparations, excluding sex hormones and insulins drugs
hormonal = pd.read_csv('../data/drug interactions/ddinter_downloads_code_H.csv')
#Interaction involving antineoplastic and immunomodulating agents drugs
immuno = pd.read_csv('../data/drug interactions/ddinter_downloads_code_L.csv')
#Interaction involving antiparasitic products, insecticides and repellents drugs
para = pd.read_csv('../data/drug interactions/ddinter_downloads_code_P.csv')
#Interaction involving respiratory system drugs
respiratory = pd.read_csv('../data/drug interactions/ddinter_downloads_code_R.csv')
#Interaction involving various drugs
various = pd.read_csv('../data/drug interactions/ddinter_downloads_code_V.csv')


#chemical structures of drugs
structures = pd.read_csv('../data/drug similarities/structure_links.csv')

**i. DDI dataset (interactions)**<br>
We want a binarized target representing known and unknown interactions, drug_a, drug_b and the type of DDI involved.

In [3]:
alimentary.shape, blood.shape, derma.shape, hormonal.shape, immuno.shape, para.shape, respiratory.shape, various.shape

((56367, 5),
 (15140, 5),
 (25681, 5),
 (11727, 5),
 (65389, 5),
 (5492, 5),
 (30563, 5),
 (12024, 5))

In [4]:
alimentary.head()

,DDInterID_A,Drug_A,DDInterID_B,Drug_B,Level
0,DDInter1263,Naltrexone,DDInter1,Abacavir,Moderate
1,DDInter1,Abacavir,DDInter1348,Orlistat,Moderate
2,DDInter58,Aluminum hydroxide,DDInter582,Dolutegravir,Major
3,DDInter112,Aprepitant,DDInter582,Dolutegravir,Minor
4,DDInter138,Attapulgite,DDInter582,Dolutegravir,Major


In [5]:
#labeling each type of interaction
types = {
    'alimentary': 'alimentary tract and metabolism',
    'blood': 'blood and blood forming organs',
    'derma': 'dermatological',
    'hormonal': 'systemic hormonal preparations',
    'immuno': 'antineoplastic and immunomodulating agents',
    'para': 'antiparasitic products, insecticides and repellents',
    'respiratory': 'respiratory',
    'various': 'various'
}

for key, value in types.items():
    exec(f'{key}["type"] = "{value}"')

#concatenating all drug types into 1 df that will represent all DDIs
interactions = pd.concat([alimentary, blood, derma, hormonal, immuno, para, respiratory, various]).reset_index(drop = True)

In [6]:
interactions.head()

,DDInterID_A,Drug_A,DDInterID_B,Drug_B,Level,type
0,DDInter1263,Naltrexone,DDInter1,Abacavir,Moderate,alimentary tract and metabolism
1,DDInter1,Abacavir,DDInter1348,Orlistat,Moderate,alimentary tract and metabolism
2,DDInter58,Aluminum hydroxide,DDInter582,Dolutegravir,Major,alimentary tract and metabolism
3,DDInter112,Aprepitant,DDInter582,Dolutegravir,Minor,alimentary tract and metabolism
4,DDInter138,Attapulgite,DDInter582,Dolutegravir,Major,alimentary tract and metabolism


In [7]:
#no null values
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222383 entries, 0 to 222382
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   DDInterID_A  222383 non-null  object
 1   Drug_A       222383 non-null  object
 2   DDInterID_B  222383 non-null  object
 3   Drug_B       222383 non-null  object
 4   Level        222383 non-null  object
 5   type         222383 non-null  object
dtypes: object(6)
memory usage: 10.2+ MB


In [ ]:
#removing unnecessary IDs
interactions = interactions.drop(columns = ['DDInterID_A', 'DDInterID_B'])
interactions.columns = interactions.columns.str.lower()

In [ ]:
#binarizing 'level' to represent 1 for an interaction, 0 for no interaction
interactions['level'] = interactions['level'].map({'Unknown':0, 'Minor': 1, 'Moderate': 1, 'Major': 1})
interactions = interactions.rename(columns = {'level': 'interaction'})

In [12]:
interactions['interaction'].value_counts(normalize = True)

1    0.787835
0    0.212165
Name: interaction, dtype: float64

In [13]:
interactions.head()

,drug_a,drug_b,interaction,type
0,Naltrexone,Abacavir,1,alimentary tract and metabolism
1,Abacavir,Orlistat,1,alimentary tract and metabolism
2,Aluminum hydroxide,Dolutegravir,1,alimentary tract and metabolism
3,Aprepitant,Dolutegravir,1,alimentary tract and metabolism
4,Attapulgite,Dolutegravir,1,alimentary tract and metabolism


### 2. Known drugs dataset

Only interested with drugs that have interaction (1). Found 1902 unique drugs that are involved in the dataset.

In [14]:
known_drugs = interactions[interactions['interaction'] == 1].reset_index(drop = True).drop(columns = ['interaction'])

In [15]:
known_drugs.shape

(175201, 3)

In [16]:
known_drugs = known_drugs.drop_duplicates(subset= ['drug_a', 'drug_b'])

In [17]:
known_drugs.shape

(130422, 3)

In [18]:
#drug names will be in lower case
for x in known_drugs.columns:
    known_drugs[x] = known_drugs[x].str.lower()

#### Number of unique drugs in known_drugs

In [20]:
def node_list(df):
    node_list = []
    for col in df.columns:
        for node in df[col]:
            node_list.append(node)
            
    #removing duplicates
    node_list = list(set(node_list))
    return node_list

In [ ]:
#creating list of all drugs found in DDIs
all_drugs = node_list(known_drugs[['drug_a', 'drug_b']])
all_drugs = pd.DataFrame(all_drugs, columns = ['drug_name'])

In [23]:
all_drugs.head()

,drug_name
0,icatibant
1,vibrio cholerae cvd 103-hgr strain live antige...
2,erythropoietin
3,chorionic gonadotropin (human)
4,loratadine


In [24]:
#1902 unique drugs in our interaction dataset
all_drugs.shape

(1902, 1)

**ii. Exploring naming convention of drugs from known_drugs**<br>
Some of the names have annotation describing the route of administration i.e. physostigmine and physostigmine (ophthalmic) meaning the same drug but applied to the eye. Does the route of administration affect the interactions of the drug? Should they be removed?<br>

**Findings:** If I get rid of the annotated versions i wont lose much information as they either share most of the same interactions as the regular version of the drug or don't contain many DDIs.

In [25]:
#example of the annnotation
all_drugs[all_drugs['drug_name'].str.contains('physostigmine')]

,drug_name
983,physostigmine (ophthalmic)
1238,physostigmine


In [ ]:
#complete list of the drugs
ad = all_drugs.copy()

#complete list of the drugs
#flagging drugs with the () notation
ad['flag'] = ad['drug_name'].str.find('(')

#drugs that aren't flagged with -1 are True
ad = ad[ad['flag'] != -1]

In [29]:
ad.head()

,drug_name,flag
1,vibrio cholerae cvd 103-hgr strain live antige...,48
3,chorionic gonadotropin (human),23
6,octasulfur (topical),11
19,medrysone (ophthalmic),10
52,fluticasone (nasal),12


In [30]:
#seperating nomenclature with '()'
ad['drug_name'] = ad['drug_name'].str.split('(')
ad['annotation'] = ad['drug_name'].apply(lambda x: x[1][:-1])
ad['drug_name'] = ad['drug_name'].apply(lambda x: x[0])

In [31]:
#list of the special annotations
ad['annotation'].value_counts()[:10]

topical                     84
ophthalmic                  51
nasal                       19
liposomal                    6
liposome                     3
human                        3
formaldehyde inactivated     2
otic                         2
recombinant                  2
sodium                       2
Name: annotation, dtype: int64

In [ ]:
#known DDIs with ophthalmic version of Physostigmine
ophthalmic = known_drugs[known_drugs['drug_b'] == 'physostigmine (ophthalmic)']
#known DDis with regular Physostigmine
regular = known_drugs[known_drugs['drug_b'] == 'physostigmine']

In [34]:
#ophthalmic doesn't seem to have as many DDIs as the regular version of Physostigmine
ophthalmic.shape, regular.shape

((49, 3), (81, 3))

In [35]:
#the ophthalmic version only has 1 different drug interaction to the regular
#this means it doesn't have much of an impact to remove the ophthalmic version
set(ophthalmic['drug_a']) - set(regular['drug_a'])

{'ipratropium'}

In [36]:
#theres only 1 instance of Benzalkonium (topical)
known_drugs[known_drugs['drug_b'].str.contains('benzalkonium')]

,drug_a,drug_b,type
70228,hyaluronic acid,benzalkonium (topical),dermatological


In [37]:
#there are only 2 instances of Dinoprostone (topical)
known_drugs[known_drugs['drug_a'].str.contains('dinoprostone')]

,drug_a,drug_b,type
19301,dinoprostone (topical),misoprostol,alimentary tract and metabolism
46825,dinoprostone (topical),urea,blood and blood forming organs


## Latent factors
We will be referencing collaborative filtering methodology which relies on the assumption that similar drugs will have similar interactions. These 'representations' of the drugs will be another aspect of their profiles. This requires latent features that are 'representations' of the same information given by the DDIs, and their respective drugs. These will be used as indications of the relationships between drugs. Examples of these derive from the profiles of these drugs that would lead to a DDI i.e. side effects, metabolic pathways, targets, and chemical structures. I've chosen chemical structures to represent the latent space, and will create a similarity score between every drug. 

### 1. Feature engineering Drug Similarities using Chemical Structures


The chemical structure notation are written in following the Simplified Molecular Input Line Entry System (SMILES). Some rules to account for:<br>
- All elements of the periodic table can be represented with their letters, further clarification can be made by putting the element within square brackets []
- Bonds are represented as -,=,#,*,.
- Entire branches are denoted within parenthesis ()
- Carbons are annotated with with a number to indicate its position i.e. C1, C2...
- Charged atoms are denoted with {}
- Tetrahedral centers indicated by @



**

**i. Cleaning Structures dataset**

In [38]:
structures.columns = structures.columns.str.lower().str.replace(' ', '_')

In [39]:
structures.head(2)

,name,smiles,inchikey,inchi,formula
0,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,OIRCOABEOLEUMC-GEJPAHFPSA-N,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,C98H138N24O33
1,Leuprolide,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,GFIJNRVAKGFPGQ-LIJARHBVSA-N,InChI=1S/C59H84N16O12/c1-6-63-57(86)48-14-10-2...,C59H84N16O12


In [40]:
structures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11912 entries, 0 to 11911
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      11912 non-null  object
 1   smiles    11297 non-null  object
 2   inchikey  11300 non-null  object
 3   inchi     11300 non-null  object
 4   formula   11302 non-null  object
dtypes: object(5)
memory usage: 465.4+ KB


In [41]:
#relevant columns
structures= structures[['name', 'smiles']]
#removing null values
structures = structures.dropna(axis = 0)

In [42]:
structures.isna().sum()

name      0
smiles    0
dtype: int64

**ii. Merging chemical structures to the drugs found in all_drugs**<br>
Resulting from the merge, the amount of unique drugs were reduced from 1910 to 1404. This was translated into the original known_drugs DDI dataset.

In [43]:
#preparing for merge chemical structures to the relevant drugs in all_drugs
structures = structures.rename(columns = {'name': 'drug_name'})

In [44]:
#drug_names will be lower case
all_drugs['drug_name'] = all_drugs['drug_name'].str.lower()
structures['drug_name'] = structures['drug_name'].str.lower()      

In [45]:
all_drugs.shape

(1902, 1)

In [46]:
chem_sim = pd.merge(all_drugs, structures, on= 'drug_name', how='inner')

In [47]:
#the resulting merge reduced individual drugs from 1902 to 1413
chem_sim.shape

(1413, 2)

In [48]:
c = node_list(chem_sim[['drug_name']])
len(c)

1413

**iii. Address mismatch in unique drugs of known_drugs vs chem_sim**

In [49]:
#new list of drugs that we focus on
all_drugs_modified = chem_sim[['drug_name']]

In [50]:
#list of all of the drugs where we don't have chemical structure data on
dropped = list(set(all_drugs['drug_name']) - set(all_drugs_modified['drug_name']))
len(dropped)

489

In [51]:
dropped[:5]

['castor oil',
 'laronidase',
 'secretin human',
 'vibrio cholerae cvd 103-hgr strain live antigen (live)',
 'asparaginase escherichia coli']

In [52]:
#dropping all interactions that have drugs that don't appear in new drug list
known_drugs.shape

(130422, 3)

In [53]:
all_drugs_modified = list(chem_sim['drug_name'])

In [54]:
#modified dataset of known DDIs excluding the drugs that were dropped
known_drugs_modified = known_drugs.copy()
known_drugs_modified = known_drugs_modified[known_drugs_modified['drug_a'].isin(all_drugs_modified)]
known_drugs_modified = known_drugs_modified[known_drugs_modified['drug_b'].isin(all_drugs_modified)]

In [55]:
#still have 89008 DDIs
known_drugs_modified.shape

(89008, 3)

**iv. address the further reduction from exception interactions**<br>
Some interactions that were dropped held unique drugs that only had 1 interaction, resulting in the loss of that unique drug. 

In [56]:
len(all_drugs_modified)

1413

In [57]:
#the filtering has further reduced the number of unique drugs. why?
after_culling = node_list(known_drugs_modified[['drug_a', 'drug_b']])
len(after_culling)

1404

In [58]:
all_drugs = node_list(known_drugs_modified[['drug_a', 'drug_b']])
set(c) - set(all_drugs)

{'cromoglicic acid',
 'etidocaine',
 'hyaluronic acid',
 'mepivacaine',
 'migalastat',
 'miglustat',
 'nedocromil',
 'pyrophosphoric acid',
 'zanamivir'}

In [59]:
#reason for reduced unique drugs is that the drugs in the above list are solely involved with a drug that was removed
#this deems them unimportant
known_drugs[known_drugs['drug_b'] == 'cromoglicic acid']

,drug_a,drug_b,type
17634,"insulin human (inhalation, rapid acting)",cromoglicic acid,alimentary tract and metabolism


In [60]:
#DDIs with 1404 unique drugs
known_drugs= known_drugs_modified
all_drugs = pd.DataFrame(all_drugs, columns = ['drug_name'])

**v. one more time**

In [61]:
#further aligning of chem_sim drug portfolio to the modified known_drugs drug portfolio
chem_sim = chem_sim[chem_sim['drug_name'].isin(list(all_drugs['drug_name']))]

### 2. Tokenisation of SMILES

**Seperating the chemical structure of the drugs whilst retaining relevant characteristics in notation**<br>
The SMILES notation needs to be tokenised appropriately in to account for substructures expressed in the notation. There are already many different tokenisers available online for this task. Instead of the basic versions of SMILES tokenisers, I've decided to use one that has been pre-trained from ChEMBL data (another database that uses the same SMILES notation) from SmilesPE.

**i. Applying pre-trained SMILES tokenizer: SmilesPE**

In [62]:
#opening vocab list
spe_vocab = codecs.open('../resources/SPE_ChEMBL.txt')
#initialising tokenizer
spe = SPE_Tokenizer(spe_vocab)

In [63]:
c = chem_sim.copy()
#applying tokenization to smiles and appending to tokens list
tokens = []
for x in c['smiles']:
    tokens.append(spe.tokenize(x))

In [64]:
#adding tokens to new column, splitting spaces so that each entry is a list of tokens
chem_sim['tokens'] = tokens
chem_sim['tokens'] = chem_sim['tokens'].map(lambda x: x.split(' '))

In [65]:
#the tokenization has been successful in keeping characteristics of each element in the chemical structure
chem_sim.head()

,drug_name,smiles,tokens
0,icatibant,[H][C@]12C[C@]([H])(N(C(=O)[C@@]3([H])CC4=C(CN...,"[[H], [C@]12C, [C@](, [H], ), (, N(C(=O), [C@@..."
1,loratadine,CCOC(=O)N1CCC(CC1)=C1C2=C(CCC3=C1N=CC=C3)C=C(C...,"[CCO, C(=O)N1CCC(, CC1), =C1, C2=C(, CCC3, =C1..."
2,boceprevir,[H][C@]12CN([C@H](C(=O)NC(CC3CCC3)C(=O)C(N)=O)...,"[[H], [C@]12, CN(, [C@H](, C(=O)NC(, CC3, CCC3..."
3,levacetylmethadol,CC[C@H](OC(C)=O)C(C[C@H](C)N(C)C)(C1=CC=CC=C1)...,"[CC[C@H](, OC(C)=O), C(, C[C@H](C), N(C)C), (,..."
4,imipramine,CN(C)CCCN1C2=CC=CC=C2CCC2=CC=CC=C12,"[CN(C), CCCN1, C2=, CC=, CC, =C2, CCC2, =, CC=..."


In [66]:
#saving this version of chem_sim for eda later on
chem_sim.to_csv('../data/cleaned data/chem_sim_eda.csv', index = False)

In [67]:
chem_sim = chem_sim.drop(columns = ['smiles'])

**Label encoding tokens. (Pre-processing for modeling)**<br>
Had to be performed in this notebook rather than the modeling section because it wouldn't work in the modeling notebook for some odd reason.

In [68]:
labelencoder = LabelEncoder()

In [69]:
#exploding tokens column to then label them into numerical data
encode = chem_sim[['tokens']]
encode = encode.explode('tokens')
#label encoding
encode['labels'] = labelencoder.fit_transform(encode['tokens'])
encode = encode.reset_index()
#groupby the index and aggregate to a list. output are the label encoded features in 1 list
encode = encode.groupby('index').agg(list)
chem_sim['labels'] = encode['labels']

In [70]:
chem_sim.head()

,drug_name,tokens,labels
0,icatibant,"[[H], [C@]12C, [C@](, [H], ), (, N(C(=O), [C@@...","[917, 889, 880, 917, 59, 31, 594, 830, 917, 59..."
1,loratadine,"[CCO, C(=O)N1CCC(, CC1), =C1, C2=C(, CCC3, =C1...","[491, 158, 369, 106, 287, 412, 106, 634, 324, ..."
2,boceprevir,"[[H], [C@]12, CN(, [C@H](, C(=O)NC(, CC3, CCC3...","[917, 888, 520, 839, 167, 382, 413, 146, 226, ..."
3,levacetylmethadol,"[CC[C@H](, OC(C)=O), C(, C[C@H](C), N(C)C), (,...","[510, 690, 139, 572, 599, 31, 258, 388, 388, 2..."
4,imipramine,"[CN(C), CCCN1, C2=, CC=, CC, =C2, CCC2, =, CC=...","[523, 449, 286, 388, 324, 108, 410, 92, 388, 3..."


In [71]:
chem_sim.to_csv('../data/cleaned data/chem_sim_vectors.csv', index = False)

### Creating nodes representing DDI

In [72]:
known_list = node_list(known_drugs)

In [73]:
len(known_list)

1412

In [74]:
#saving version of known_drugs with the names and types before converting into numerical nodes
known_drugs.to_csv('../data/cleaned data/known_drugs_names.csv', index = False)

In [75]:
#applying numerical IDs for each drug that represent nodes
known_drugs= known_drugs[['drug_a', 'drug_b']]
nodes = known_drugs
nodes = nodes.stack().rank(method='dense').astype(int).unstack()

In [76]:
#making nodes start from 0
nodes  = nodes.apply(lambda x: x-1)
nodes = nodes.rename(columns = {'drug_a': 'druga_id', 'drug_b': 'drugb_id'})

In [77]:
known_drugs = pd.concat([known_drugs, nodes], axis = 1)

In [78]:
known_drugs = known_drugs.sort_values(by = 'druga_id', ascending = True).reset_index(drop = True)

In [79]:
known_drugs.head()

,drug_a,drug_b,druga_id,drugb_id
0,abacavir,cabozantinib,0,174
1,abacavir,orlistat,0,945
2,abacavir,cobicistat,0,295
3,abacavir,trabectedin,0,1305
4,abacavir,teriflunomide,0,1257


### Drug_id lookup

In [80]:
#list of all unique drugs in each row of drug_a and drug_b
druga_lookup = known_drugs[['druga_id', 'drug_a']].drop_duplicates(subset = 'drug_a').reset_index(drop = True)
drugb_lookup = known_drugs[['drugb_id', 'drug_b']].drop_duplicates(subset = 'drug_b').reset_index(drop = True)

In [81]:
#drug b has more individual IDs 
druga_lookup.shape, drugb_lookup.shape

((1307, 2), (1361, 2))

In [82]:
druga_lookup.head()

,druga_id,drug_a
0,0,abacavir
1,1,abarelix
2,2,abemaciclib
3,3,abiraterone
4,4,acalabrutinib


In [83]:
#cross checking that the assigned nodes are the same for both columns
drugb_lookup = drugb_lookup.sort_values(by = 'drugb_id', ascending = True)
drugb_lookup.head()

,drugb_id,drug_b
1202,0,abacavir
1360,1,abarelix
825,2,abemaciclib
604,3,abiraterone
219,4,acalabrutinib


In [84]:
#merging the drug_a and drug_b lookups since drug_b has other unique drugs
drugb_lookup = drugb_lookup.rename(columns = {'drugb_id': 'druga_id'})

In [85]:
all_drugs_lookup = pd.merge(drugb_lookup, druga_lookup, on = 'druga_id', how = 'outer')

In [86]:
all_drugs_lookup['drug_b'] = all_drugs_lookup['drug_b'].fillna(all_drugs_lookup['drug_a'])

In [87]:
all_drugs_lookup = all_drugs_lookup.drop(columns = ['drug_a'])
all_drugs_lookup = all_drugs_lookup.rename(columns = {'drug_b': 
                                                     'drug_name'})

In [88]:
#complete list of the drugs and their associated nodes
all_drugs_lookup = all_drugs_lookup.sort_values(by = 'druga_id', ascending = True).reset_index(drop = True)

In [89]:
all_drugs_lookup.head()

,druga_id,drug_name
0,0,abacavir
1,1,abarelix
2,2,abemaciclib
3,3,abiraterone
4,4,acalabrutinib


In [90]:
#we want just 2 columns representing 2 nodes
known_drugs = known_drugs.drop(columns = ['drug_a', 'drug_b'])
known_drugs.head()

,druga_id,drugb_id
0,0,174
1,0,945
2,0,295
3,0,1305
4,0,1257


In [91]:
all_drugs_lookup.to_csv('../data/cleaned data/all_drugs_lookup.csv', index = False)
interactions.to_csv('../data/cleaned data/interactions.csv', index = False)
known_drugs.to_csv('../data/cleaned data/known_drugs.csv', index = False)
chem_sim.to_csv('../data/cleaned data/chem_sim.csv', index = False)